# Kaggle: COVID-19 Forecast
https://www.kaggle.com/c/covid19-global-forecasting-week-3/data

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact
import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore")

## Import data

### From Kaggle: confirmed cases and fatalities

In [57]:
filename = 'data/train.csv'
df_train_raw = pd.read_csv(filename)

In [61]:
df_train_raw.head(5)

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


### Demographics

In [62]:
filename = 'data/demographics.csv'
df_demo_raw = pd.read_csv(filename)

In [63]:
df_demo_raw.head(5)

,country,country_code,series,value
0,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,86.0007531
1,Afghanistan,AFG,"Age dependency ratio, old",4.750925407
2,Afghanistan,AFG,"Age dependency ratio, young",81.24982769
3,Afghanistan,AFG,"Age population, age 00, female, interpolated",549973
4,Afghanistan,AFG,"Age population, age 00, male, interpolated",581860


### Human Development Index

In [64]:
filename = 'data/hdi.csv'
df_hdi_raw = pd.read_csv(filename)

In [65]:
df_hdi_raw.head(5)

,country,hdi,skilled_employ_ratio,physicians_per_10k,hospital_beds_per_10k,vulnerable_employment_perc_emp
0,Norway,0.953688337,14.44329897,46.336,39,4.8
1,Switzerland,0.945936117,12.9742268,42.363,47,9
2,Ireland,0.942472821,5.185185185,30.861,28,10.9
3,Germany,0.938784706,5.791743119,42.087,83,5.9
4,"Hong Kong, China (SAR)",0.93880915,1.926988266,..,..,5.9


## Data Preparation

### Select countries

In [66]:
# select_countries = ['Australia', 'Brazil', 'France', 'Germany', 'Italy', 'Japan', 'Korea, South',
#                     'Mexico', 'India', 'Spain', 'South Africa', 'United Kingdom', 'Turkey']

select_countries = ['Brazil', 'France', 'Germany', 'Italy', 'Japan', 'Korea, South',
                    'Spain', 'United Kingdom']

select_demo_series = ['Death rate, crude (per 1,000 people)', 'Population, total', 
                      'Population ages 0-14, total', 'Population ages 15-64, total', 'Population ages 65 and above, total']

In [67]:
flag = df_train_raw['Country_Region'].apply(lambda x: x in select_countries)
df_train = df_train_raw[flag].copy()

In [68]:
flag_country = df_demo_raw['country'].apply(lambda x: x in select_countries)
flag_series = df_demo_raw['series'].apply(lambda x: x in select_demo_series)
df_demo = df_demo_raw[(flag_country) & (flag_series)].copy()
df_demo.drop('country_code', axis=1, inplace=True)

In [69]:
flag = df_hdi_raw['country'].apply(lambda x: x in select_countries)
df_hdi = df_hdi_raw[flag].copy()

In [70]:
cols = list(df_hdi.drop('country', axis=1).columns)
df_hdi[cols] = df_hdi[cols].astype('float64')

### Pivot `df_demo` series

In [71]:
df_demo.head()

,country,series,value
4781,Brazil,"Death rate, crude (per 1,000 people)",6.378
4809,Brazil,"Population ages 0-14, total",45066349
4827,Brazil,"Population ages 15-64, total",144882359
4869,Brazil,"Population ages 65 and above, total",17885124
4891,Brazil,"Population, total",207833831


In [72]:
df_demo = pd.pivot_table(data=df_demo, index='country', columns='series', values='value', aggfunc='sum')
delete_cols = df_demo.columns

In [73]:
df_demo.head()

series,"Death rate, crude (per 1,000 people)","Population ages 0-14, total","Population ages 15-64, total","Population ages 65 and above, total","Population, total"
country,,,,,
Brazil,6.378,45066349,144882359,17885124,207833831
France,9,12109162,41604279,13151703,66865144
Germany,11.3,11117936,53874035,17665031,82657002
Italy,10.7,8157364,38758774,13620571,60536709
Japan,10.8,16246827,76168000,34370970,126785797


In [74]:
df_demo['population'] = df_demo['Population, total'].apply(int)
df_demo['death_rate_1000'] = df_demo['Death rate, crude (per 1,000 people)'].apply(float)
df_demo['population_0_14'] = df_demo['Population ages 0-14, total'].apply(int)/df_demo['population']
df_demo['population_15_64'] = df_demo['Population ages 15-64, total'].apply(int)/df_demo['population']
df_demo['population_65_above'] = df_demo['Population ages 65 and above, total'].apply(int)/df_demo['population']
df_demo.drop(delete_cols, axis=1, inplace=True)

In [75]:
df_demo.head(10)

series,population,death_rate_1000,population_0_14,population_15_64,population_65_above
country,,,,,
Brazil,207833831,6.378,0.216838,0.697107,0.086055
France,66865144,9.000,0.181098,0.622212,0.196690
Germany,82657002,11.300,0.134507,0.651778,0.213715
Italy,60536709,10.700,0.134751,0.640252,0.224997
Japan,126785797,10.800,0.128144,0.600761,0.271095
"Korea, South",51466201,5.600,0.132255,0.729223,0.138522
Spain,46593236,9.000,0.147033,0.661508,0.191458
United Kingdom,66058859,9.200,0.176350,0.640739,0.182911


### Aggregate cases and deaths by country

In [76]:
df = df_train.drop('Id', axis=1).groupby(by=['Country_Region', 'Date'], as_index=False).sum()
df.head()

,Country_Region,Date,ConfirmedCases,Fatalities
0,Brazil,2020-01-22,0.0,0.0
1,Brazil,2020-01-23,0.0,0.0
2,Brazil,2020-01-24,0.0,0.0
3,Brazil,2020-01-25,0.0,0.0
4,Brazil,2020-01-26,0.0,0.0


### Check date of first case

In [77]:
threshold = 100
flag = df['ConfirmedCases'] >= threshold
first_case = df[flag].groupby(by=['Country_Region']).min()[['Date']]
first_case.rename(columns={'Date':'first_case'}, inplace=True)

### Include first date in `df` and calculate `days`  (elapsed days)

In [78]:
df2 = pd.merge(df, first_case, how='left', left_on='Country_Region', right_on='Country_Region')

In [79]:
df2['Date'] = pd.to_datetime(df2['Date'])
df2['first_case'] = pd.to_datetime(df2['first_case'])
df2['days'] = df2['Date'] - df2['first_case']
df2['days'] = df2['days'].apply(lambda x: x.days)

In [80]:
flag = df2['days'] >= 0
df = df2[flag].copy()
df.head()

,Country_Region,Date,ConfirmedCases,Fatalities,first_case,days
51,Brazil,2020-03-13,151.0,0.0,2020-03-13,0
52,Brazil,2020-03-14,151.0,0.0,2020-03-13,1
53,Brazil,2020-03-15,162.0,0.0,2020-03-13,2
54,Brazil,2020-03-16,200.0,0.0,2020-03-13,3
55,Brazil,2020-03-17,321.0,1.0,2020-03-13,4


### Calculate `diff_cases` and `diff_deaths`

In [81]:
df['diff_cases'] = df['ConfirmedCases'].diff(periods=1)
df['diff_deaths'] = df['Fatalities'].diff(periods=1)
mask = df['Country_Region'] != df['Country_Region'].shift(1)
df['diff_cases'][mask] = np.nan
df['diff_deaths'][mask] = np.nan

In [82]:
df.head()

,Country_Region,Date,ConfirmedCases,Fatalities,first_case,days,diff_cases,diff_deaths
51,Brazil,2020-03-13,151.0,0.0,2020-03-13,0,NaN,NaN
52,Brazil,2020-03-14,151.0,0.0,2020-03-13,1,0.0,0.0
53,Brazil,2020-03-15,162.0,0.0,2020-03-13,2,11.0,0.0
54,Brazil,2020-03-16,200.0,0.0,2020-03-13,3,38.0,0.0
55,Brazil,2020-03-17,321.0,1.0,2020-03-13,4,121.0,1.0


### Join dataframes: `df`, `df_hdi`, `df_demo`

In [83]:
df2 = pd.merge(df, df_hdi, how='left', left_on='Country_Region', right_on='country')
df3 = pd.merge(df2, df_demo, how='left', left_on='Country_Region', right_on='country')
df3.head()

,Country_Region,Date,ConfirmedCases,Fatalities,first_case,days,diff_cases,diff_deaths,country,hdi,skilled_employ_ratio,physicians_per_10k,hospital_beds_per_10k,vulnerable_employment_perc_emp,population,death_rate_1000,population_0_14,population_15_64,population_65_above
0,Brazil,2020-03-13,151.0,0.0,2020-03-13,0,NaN,NaN,Brazil,0.761153,1.349538,21.499,22.0,27.6,207833831,6.378,0.216838,0.697107,0.086055
1,Brazil,2020-03-14,151.0,0.0,2020-03-13,1,0.0,0.0,Brazil,0.761153,1.349538,21.499,22.0,27.6,207833831,6.378,0.216838,0.697107,0.086055
2,Brazil,2020-03-15,162.0,0.0,2020-03-13,2,11.0,0.0,Brazil,0.761153,1.349538,21.499,22.0,27.6,207833831,6.378,0.216838,0.697107,0.086055
3,Brazil,2020-03-16,200.0,0.0,2020-03-13,3,38.0,0.0,Brazil,0.761153,1.349538,21.499,22.0,27.6,207833831,6.378,0.216838,0.697107,0.086055
4,Brazil,2020-03-17,321.0,1.0,2020-03-13,4,121.0,1.0,Brazil,0.761153,1.349538,21.499,22.0,27.6,207833831,6.378,0.216838,0.697107,0.086055


### Calculate `cases_10k`, `diff_cases_10k`, `deaths_10k`, `diff_deaths_10k`

In [84]:
df3['cases_10k']       = df3['ConfirmedCases'] / df3['population'] * 10000
df3['diff_cases_10k']  = df3['diff_cases'] / df3['population'] * 10000
df3['deaths_10k']      = df3['Fatalities'] / df3['population'] * 10000
df3['diff_deaths_10k'] = df3['diff_deaths'] / df3['population'] * 10000
df3['cases_physician'] = df3['cases_10k'] / df3['physicians_per_10k']
df3['deaths_physician'] = df3['deaths_10k'] / df3['physicians_per_10k']

In [85]:
df3.head()

,Country_Region,Date,ConfirmedCases,Fatalities,first_case,days,diff_cases,diff_deaths,country,hdi,...,death_rate_1000,population_0_14,population_15_64,population_65_above,cases_10k,diff_cases_10k,deaths_10k,diff_deaths_10k,cases_physician,deaths_physician
0,Brazil,2020-03-13,151.0,0.0,2020-03-13,0,NaN,NaN,Brazil,0.761153,...,6.378,0.216838,0.697107,0.086055,0.007265,NaN,0.000000,NaN,0.000338,0.000000
1,Brazil,2020-03-14,151.0,0.0,2020-03-13,1,0.0,0.0,Brazil,0.761153,...,6.378,0.216838,0.697107,0.086055,0.007265,0.000000,0.000000,0.000000,0.000338,0.000000
2,Brazil,2020-03-15,162.0,0.0,2020-03-13,2,11.0,0.0,Brazil,0.761153,...,6.378,0.216838,0.697107,0.086055,0.007795,0.000529,0.000000,0.000000,0.000363,0.000000
3,Brazil,2020-03-16,200.0,0.0,2020-03-13,3,38.0,0.0,Brazil,0.761153,...,6.378,0.216838,0.697107,0.086055,0.009623,0.001828,0.000000,0.000000,0.000448,0.000000
4,Brazil,2020-03-17,321.0,1.0,2020-03-13,4,121.0,1.0,Brazil,0.761153,...,6.378,0.216838,0.697107,0.086055,0.015445,0.005822,0.000048,0.000048,0.000718,0.000002


## Export consolidated database

In [86]:
filename = 'output/consolidated_db.csv'
df3.to_csv(filename, index=False)

## Exploratory Data

In [92]:
@interact(metric=df3.columns)
def explore(metric='diff_cases'):
    plt.figure(figsize=(12,8))
    for c in df3['country'].unique():
        mask = df3['country'] == c
        plt.plot(df3['Date'][mask], df3[metric][mask], label=c)
    plt.legend()

interactive(children=(Dropdown(description='metric', index=6, options=('Country_Region', 'Date', 'ConfirmedCas…

In [88]:
@interact(metric=df3.columns)
def explore_hbar(metric='diff_cases'):
    plt.figure(figsize=(12,8))
    mask = df3['Date'] == '2020-04-02'
    plt.bar(x=df3['country'].unique(), height=df3[metric][mask].apply(float))
    plt.xticks(rotation=90)

interactive(children=(Dropdown(description='metric', index=6, options=('Country_Region', 'Date', 'ConfirmedCas…

In [91]:
df_hdi

,country,hdi,skilled_employ_ratio,physicians_per_10k,hospital_beds_per_10k,vulnerable_employment_perc_emp
3,Germany,0.938785,5.791743,42.087,83.0,5.9
14,United Kingdom,0.920352,5.748646,28.058,28.0,13.0
18,Japan,0.914696,3.540948,24.118,134.0,8.4
22,"Korea, South",0.905832,2.935321,23.661,115.0,23.5
24,Spain,0.892788,2.588969,40.691,30.0,11.3
26,France,0.891057,4.652125,32.349,65.0,7.4
28,Italy,0.882584,3.293292,40.931,34.0,17.0
79,Brazil,0.761153,1.349538,21.499,22.0,27.6
